## Summary of what we are looking at 
- how we define success. 
- In most cells, give a breif descrition of what we are doing and were files come from.
- have one cell that is markdown, then code cell
- combine cells
- describe visualization. What is the take away, what is the message.
- will have to make some judgement calls on what we could leave out.
- dont need to explain code for vidualization, just what the visualization says.

# INTRO!

In [ ]:
#importing of all of the things

In [1]:
# importing of all the data
# explain where this data came from

# Alexis' Work
- age group vs imdb rating scatter plot
- genre vs imdb rating set of histograms

### Do movies targeted at different age groups perform differently?
The two things that she made were two ways to look at that

### Do different generas of movies perform differntly?

### Do movies that earn more get better rated?

### Are movies on different streaming platforms get better rated?

### Are movies on steaming platofroms better?

# Ben's Work

### Do better rated movies win more awardsscatterplot compairing imdb ratings and rotton tomatoes

### How has the funding of movies increased over time?

### How have ratings of movies change over time?

### Movies budget vs. rating

# Cat's Work

### Do better rated movies receive more awards/nominations?